<a href="https://colab.research.google.com/github/jazkre/01RAD/blob/main/assignments/2022/assignment01/01RAD2022_assignment01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  1. zápočtová úloha z 01RAD 2022/23

## Popis úlohy

V tomto úkolu je cílem provést předzpracování datového souboru, jeho vizualizaci a jednoduchou lineární regresní úlohu, kde se budeme zajímat o ceny nemovitostí. Za tímto účelem využijeme datový set *saratosa_hoouses* z knihovny *moderndive* obsahující výběr 1057 domů.



## Podmínky a body

Úkol i protokol vypracujte samostatně. Pokud na řešení nějaké úlohy budete přesto s někým spolupracovat, radit se, nezapomeňte to u odpovědi na danou otázku uvést. Tato zápočtová úloha obsahuje 10 otázek po 1 bodu. Celkem za 3 zápočtové úlohy bude možné získat 30 bodů, přičemž pro získání zápočtu je potřeba více jak 20 bodů. Další dodatečné body mohu případně individuálně udělit za řešení mini domácích úkolů z jednotlivých hodin.



## Odevzdání

Protokol ve formátu Rmd+pdf, nebo jako Jupyter notebook (idealně odkaz na gitlab s možností spustit v Colabu) nejpozději do 1. 11. 2022.



### Předzpracování dat:




In [ ]:
list_of_packages <- c("tidyverse", "MASS","GGally","moderndive")
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
missing_packages
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)

In [ ]:
? saratoga_houses

**Data**

In [ ]:
head(saratoga_houses)

## Otázka 01

Zjistěte, zdali data neobsahují chybějící hodnoty *NA*. Pokud ano, tak rozhodněte zdali můžete příslušná pozorování z dat odstranit a proč. Které proměnné jsou kvantitativní a které kvalitativní? Jeli možno některé zařadit do obou skupin, pro kterou byste se rozhodli? Které proměnné je možné pužít jako faktorové ordinální a jaké jako faktorové nominální a proč? Spočtěte základní statistiky pro jednotlivé proměnné.

### Řešení 01:

Aby sme zistili či dáta obsahujú chýbajúce hodnoty, tj. *NA* využijeme funkciu ***summary***. 
Skontrolujeme charakteristiky pre každú premennú, či sa vo výpise nachádza kategória *NA's*. Vidíme, že premenná *lot_size* postráda 9 hodnôt, čo činí 0.85% dát. Keďže ide o malý počet hodnôt a navyše chýbajú v premennej *lot_size*, teda veľkosť pozemku, pričom v datasetu pozorujeme nekonzistentnosť medzi veľkosťou obytnej plochy a pozemkom (či inými parametrami) a faktom, že ďalej vyberáme podmnožinu na základe *lot_size*, rozhodli sme sa 9 z 1057 pozorovaní odstrániť.

Za kvantitatívne premenné považujem : **price, living_area, bathrooms, bedrooms, fireplacesm lot_size, age** a za kvalitatívne **fireplace**, zároveň by sme ako kvalitatívne premenné mohli uvažovať aj **bedrooms, bathrooms** či **fireplaces** v ponímaní popisu domu podľa počtu izieb (3kk, 3+1...). Premenné by som ponechala ako kvantitatívne.

Za faktorové premenné je možné vziať **bedrooms, bathrooms, fireplaces**, pričom ako ordinálnu by sa dalo uvažovať **bedrooms** a usporiadať od najmenšieho počtu spální po najvyšší.

In [ ]:
summary(saratoga_houses)

In [ ]:
pctmiss <- colSums(is.na(saratoga_houses))/nrow(saratoga_houses)
round(pctmiss, 4)

In [ ]:
library(dplyr)

saratoga_houses %>% 
  filter(if_any(everything(), is.na))

In [ ]:
sh <- na.omit(saratoga_houses)

In [ ]:
summary(sh)

Základné štatistiky vidíme v tabuľke summary, chýbajú nám rozptyly veličín(bez veličiny fireplace), ktoré uvedieme v nasledujúcej bunke:

In [ ]:
c(var(sh$price), var(sh$living_area),var(sh$bathrooms), var(sh$bedrooms),var(sh$fireplaces),var(sh$lot_size), var(sh$age))

## Otázka 02

Chceme koupit nemovitost v zahraničí a průzkumem trhu jsme obdřeli předchozí data set *saratoga_houses*. Jelikož ale máme přesnější požadavky a nerozumíme imperiálním jednotkám, potřebujeme data upravit:

* Převeďte cenu nemovitostí z dolarů na koruny v tisících a plochu pozemku a obytnou plochu z akrů a čtverečích stop na $m^2$. (chceck describtion by *? saratoga_houses*) 
* Vyberte jen nemovitosti starší 10 let a mladší 50 let, jejichž cena je menší než 7500000 Kč, a plocha pozemku je mezi 500 a 5000 $m^2$.
* Počet koupelen a počet pokojů převeďte na faktorové proměnné o 3 úrovních.

**Dále pracujte jen s takto omezeným datasetem a s proměnnýma *cena*, *plocha_obytna*, *plocha_pozemku* *pocet_pokoju*,*stari_domu*, *pocet_koupelen*, *krb*.**

### Řešení 02:

Převeďte cenu nemovitostí z dolarů na koruny v tisících a plochu pozemku a obytnou plochu z akrů a čtverečích stop na $m^2$.

-urobíme tak pomocou balíčkov *measurements* a *priceR*:

In [ ]:
install.packages("measurements")
library(measurements)
#conv_unit(x, from, to)

install.packages("priceR")
library(priceR)
#convert_currencies(price_start,  from,  to,  date = lubridate::today(),  floor_unit = "day")

In [ ]:
sh$living_area=conv_unit(sh$living_area, "ft2","m2")
sh$lot_size=conv_unit(sh$lot_size, "acre","m2")

In [ ]:
sh$price=convert_currencies(sh$price,"USD", "CZK", date = lubridate::today(),  floor_unit = "day")

In [ ]:
sh$price=sh$price%>% round_up_to_nearest(1000)

In [ ]:
head(sh)

Vyberte jen nemovitosti starší 10 let a mladší 50 let, jejichž cena je menší než 7500000 Kč, a plocha pozemku je mezi 500 a 5000  m2:

-učiníme tak pomocou funkcie *subset* a nerovnostných podmienok:

In [ ]:
sh_sel=subset(sh, sh$age>=10 & sh$age<=50 & sh$lot_size>=500 & sh$lot_size<=5000 & sh$price<7500000)

Počet koupelen a počet pokojů převeďte na faktorové proměnné o 3 úrovních:

In [ ]:
sh_sel$pocet_koupelen_f='2-2.5'
sh_sel$pocet_koupelen_f[sh_sel$bathrooms<2]='1-1.5'
sh_sel$pocet_koupelen_f[sh_sel$bathrooms>=3]='3-3.5'
sh_sel$pocet_koupelen_f<-as.factor(sh_sel$pocet_koupelen_f)

In [ ]:
sh_sel$pocet_pokoju_f='2-3_stredny'
sh_sel$pocet_pokoju_f[sh_sel$bedrooms<2]='1_maly'
sh_sel$pocet_pokoju_f[sh_sel$bedrooms>3]='4-5_velky'
sh_sel$pocet_pokoju_f<-as.factor(sh_sel$pocet_pokoju_f)

In [ ]:
head(sh_sel)

Na záver 2.úlohy premenujeme premenné do češtiny, podľa zadania vrámci balíčka *dplyr*:

In [ ]:
sh_sel<-sh_sel%>%
  rename(cena=price,
        plocha_obytna=living_area,
        pocet_koupelen=bathrooms,
        pocet_pokoju=bedrooms,
        pocet_krbu=fireplaces,
        plocha_pozemku=lot_size,
        stari_domu=age,
        krb=fireplace)

In [ ]:
head(sh_sel)


## Otázka 03 

* Porovnejte průměry cen nemovitostí s krbem a bez krbu a otestujte, zdali na hladině významnosti $\alpha = 0.01$ je průměrná cena nemovitostí s krbem větší než průměrná cena nemovitostí bez krbu.

### Řešení 03:


Testujeme hypotézy:

$H_0 : μ_{T}> \mu_{F} $ vs. $H_1 : μ_{T} 	\leq \mu_{F} $ , na hladine významnosti $\alpha = 0.01$

*pri značení T - s krbom / F - bez krbu*

In [ ]:
install.packages("ggpubr")
library("ggpubr")

In [ ]:

ggboxplot(sh_sel, x = "krb", y = "cena", 
          color = "krb", palette = c("#00AFBB", "#E7B800"),
          ylab = "cena", xlab = "Krb")

In [ ]:
ceny_s_krbom=subset(sh_sel$cena, sh_sel$krb==TRUE)
ceny_bez_krbu=subset(sh_sel$cena, sh_sel$krb==FALSE)

In [ ]:
mean(ceny_s_krbom)
mean(ceny_bez_krbu)

In [ ]:
# Shapiro-Wilk normality test for Men's weights
with(sh_sel, shapiro.test(cena[krb==FALSE]))# p =  0.0001061 < alpha
# Shapiro-Wilk normality test for Women's weights
with(sh_sel, shapiro.test(cena[krb==TRUE])) # p = 0.007471 < alpha

p-hodnoty Shapiro-Wilkovho twstu vyšli značne menšie, než naša význačná hladina $α$, na základe čoho ďalej pokračujeme s Wilcoxonovým testom.

In [ ]:
res <- wilcox.test(cena ~ krb, data = sh_sel,
                   exact = FALSE)
res

Wilcoxon test aplikujeme podľa dokumentácie [Wilcox.test()](https://www.rdocumentation.org/packages/stats/versions/3.6.2/topics/wilcox.test), kde za alternatívnu hypotézu volíme *alternative = "greater"*, teda $H_1: μ_{F}>\mu_{T}$.

In [ ]:
wilcox.test(cena ~ krb, data = sh_sel, 
        exact = FALSE, alternative = "greater")

Na základe Wilcoxonovho testu, kde sme skúmali, $H_0$, či sú ceny nehnuteľností bez krbu v priemere nižšie než ceny nehnuteľností s krbom, sme obdržali **p-value=1** a na hladine významnosti **0.01** nemáme dosť dôkazov túto hypotézu zamietnuť.

# Vizualizace dat

## Otázka 04 

* Vykreslete scatterploty pro všechny numerické proměnné, kde bude barevně rozlišeno, zdali se jedná o nemovitost s krbem, nebo bez krbu.
*  Pro proměnné *pocet_pokoju* a *pocet_pater* a *sklep* vykreslete krabicové diagramy (nebo violin ploty), kde odezvou bude *cena*.
* Pro proměnnou *cena* vykreslete histogram spolu s jádrovým odhadem hustoty.


In [ ]:
library(ggplot2)

In [ ]:
Vykreslíme príslušné scatterploty

In [ ]:
ggplot(sh_sel, aes(x=plocha_obytna, y=cena,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

ggplot(sh_sel, aes(x=plocha_pozemku, y=cena,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

ggplot(sh_sel, aes(x=stari_domu, y=cena,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

ggplot(sh_sel, aes(x=pocet_koupelen, y=cena,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

ggplot(sh_sel, aes(x=pocet_pokoju, y=cena,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

ggplot(sh_sel, aes(x=pocet_krbu, y=cena,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

ggplot(sh_sel, aes(x=plocha_obytna, y=plocha_pozemku,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

ggplot(sh_sel, aes(x=plocha_obytna, y=stari_domu,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

ggplot(sh_sel, aes(x=plocha_obytna, y=pocet_pokoju,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

  ggplot(sh_sel, aes(x=plocha_obytna, y=pocet_koupelen,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

  ggplot(sh_sel, aes(x=plocha_obytna, y=pocet_krbu,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

  ggplot(sh_sel, aes(x=stari_domu, y=plocha_pozemku,colour = krb))+
  geom_point()+
  geom_smooth(method = lm)

Vykreslíme boxploty pre premennú počet pokojú voči cene 

In [ ]:

#wK<-ggboxplot(sh_sel, x = "pocet_pokoju", y = "cena", 
 #         color = "krb", palette = c("#00AFBB", "#E7B800"),
 #         ylab = "cena", xlab = "Pokoje")
ggboxplot(sh_sel, x = "pocet_pokoju", y = "cena", 
          #color = "krb", palette = c("#00AFBB", "#E7B800"),
          ylab = "cena", xlab = "Pokoje")

#ggarrange(wK, woK, labels = c("A", "B"),
#          common.legend = TRUE, legend = "bottom")

#ggarrange(wK, woK, 
#          ncol = 2, nrow = 1,  align = "hv", 
#          widths = c(7, 7), heights = c(2, 2),
#         common.legend = TRUE)

Premenné *pocet_pater* a *sklep* nemáme. Z dát a dokumentácie *?saratoga_houses* nie je jasné, či šíslo 2 označuje dve kúpeľne alebo kúpeľňu a 2 záchody, rovnako nemáme podklady stavebných inžinierov NYC, aby sme mohli odhadmnúť podobne ako na území SR, že na poschodie je potreba kúpeľňa, preto takýto predpoklad činiť nebudeme.

Posledným bodom teda zostáva vykresliť histogram a preložiť ho jadrovým odhadom:

In [ ]:
hist(sh_sel$cena, prob = TRUE)                             
lines(density(sh_sel$cena, bw = "sj"), col = "blue")

## Otázka 05

Pro kombinace faktorizovaných proměnných *pocet_pokoju*, *pocet_koupelen*  vykreslete cenu nemovitosti, aby bylo na obrázku vidět, jestli se v průměru liší ceny nemovitostí majících více pokojů, nebo více koupelen a zdali jsou zastoupeny všechny kombiance všech úrovních pro dvě zmíněné faktorové proměnné.

In [ ]:
head(sh_sel)

In [ ]:
sh_sel$koup_pokoj_f<-as.factor(interaction(sh_sel$pocet_koupelen_f, sh_sel$pocet_pokoju_f))

In [ ]:
rozsirene_sh<-sh_sel
levels(rozsirene_sh$koup_pokoj_f) <- c(levels(rozsirene_sh$koup_pokoj_f),"2-2.5.1_maly")
levels(rozsirene_sh$koup_pokoj_f) <- c(levels(rozsirene_sh$koup_pokoj_f),"3-3.5.1_maly")

rows <- nrow(rozsirene_sh)
 
# inserting row at end
rozsirene_sh[rows+1,] <- NA
rozsirene_sh[rows+1,ncol(rozsirene_sh)]<-"2-2.5.1_maly"

rows <- nrow(rozsirene_sh)
 
# inserting row at end
rozsirene_sh[rows+1,] <- NA
rozsirene_sh[rows+1,ncol(rozsirene_sh)]<-"3-3.5.1_maly"




In [ ]:
ggboxplot( rozsirene_sh, x = "koup_pokoj_f", y = "cena", 
          #color = "krb", palette = c("#00AFBB", "#E7B800"),
          ylab = "cena", xlab = "Pokoje a koupelny") +stat_summary(fun.y="mean", color="red", shape=15) + coord_flip()




In [ ]:
interaction(sh_sel$pocet_koupelen_f, sh_sel$pocet_pokoju_f)

## Otázka 06

Pro nemovitosti s dvěma ložnicema vykreslete závislost ceny na obytné ploše nemovitosti, kde jednotlivé události označíte barvou podle toho zdali mají krb a velikost bodů v grafu bude odpovídat počtu koupelen (pro tuto úlohu je lepší vzít počet koupelen jako numerickou proměnnou).

**Dále pracujte jen s nemovitostmi se dvěma ložnicemi.**

In [ ]:
sh_2bdr=subset(sh_sel, sh_sel$pocet_pokoju==2)

In [ ]:
ggplot(data = sh_2bdr, mapping = aes(x = plocha_obytna, y = cena, colour = krb, size = pocet_koupelen)) + 
    geom_point()



# Jednoduchý lineární model

## Otázka 07

Sestavte jednoduchý regresní model (s i bez interceptu), kde vysvětlovaná proměnná
bude cena nemovitosti a vysvětlující obytná plocha.   Spočtěte pro oba modely $R^2$ a $F$ statistiky, co nám o modelech říkají. Vyberte jeden z nich a zdůvodněte proč ho preferujete.

Na základě zvoleného modelu odpovězte, zdali cena nemovitosti závisí na obytné ploše  a pokud ano, o kolik se změní očekávaná cena pro nemovitost s obytnou plochou zvětšenou o $20 m^2$? 

## Otázka 08
Sestavte jednoduchý linární model jako v předchozí otázce pro nemovitosti s krbem a bez krbu. Jaký model vykazuje silnější linearní vztah mezi cenou a obytnou plochou? O kolik cena s rostoucí obytnou plochou pro nemovitosti s krbem roste rychleji než pro nemovitosti bez krbu?

Spočtěte 95% konfidenční intervaly pro regresní koeficienty popisující sklon regresní přímky v obou modelech a zjistěte, zdali se protínají. Co z toho můžeme vyvozovat?

Na základě těchto modelů zjistěte o kolik procent bude mít průměrná nemovitost s krbem a obytnou plochou $160m^2$ vyšší očekávanou cenu než průměrná nemovitost o stejné obytné ploše, ale bez krbu.

## Otázka 9

Vykreslete scatterplot obytné plochy a ceny nemovitostí. Do tohoto grafu vykreslete regresní přímky vybraných modelů pro nemovitosti s krbem a bez něho, jednoltivé body i regresní přímky označte barvou podle toho k jaké skupině přísluší.

Sestrojte 90% konfidenční intervaly okolo očekávaných cen pro jednoltivé skupiny a na jejich základě rozhodněte, zdali a jak se očekávané ceny budou lišit pro nemovitosti s obytnou plochou menší než $120m^2$. Je to porovnávání správné? Zdůvoněte.

## Otázka 10

Vykreslete histogramy pro rezidua modelů z předchozí otázky. Proložte je hustotou normálního rozdělení s nulovou střední hodnotou a rozptylem odpovídajícím $\hat{\sigma}^2$ z jednotlivých modelů.

Co výsledný graf říka o n našich modelech a je toto ověření dostatečné pro validaci model?

Navrněte další úpravy modelu za cílem co nejlépe predikvoat cenu nemovitosti.